In [2]:
import requests
import json

In [3]:
# ENDPOINT = "[Your endpoint of Anomaly Detector resource]/anomalydetector/v1.1"
# HEADERS = {
#     "Ocp-Apim-Subscription-Key": "[Your key of Anomaly Detector resource.]"
# }

ENDPOINT = "https://multi-ad-test-uscx.cognitiveservices.azure.com/anomalydetector/v1.1"
HEADERS = {
    "Ocp-Apim-Subscription-Key": "7fbc448065f54affa3515f2affb979d5"
}

In [28]:
API_MODEL = "{endpoint}/multivariate/models"
API_MODEL_STATUS = "{endpoint}/multivariate/models/{model_id}"
API_MODEL_BATCH_INFERENCE = "{endpoint}/multivariate/models/{model_id}:detect-batch"
API_MODEL_LAST_INFERENCE = "{endpoint}/multivariate/models/{model_id}:detect-last"
API_RESULTS = "{endpoint}/multivariate/detect-batch/{result_id}"
API_DELETE = "{endpoint}/multivariate/models/{model_id}"
DATA_SOURCE_URL= "https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_3000"

# Train a model

In [10]:
SLIDING_WINDOW = 200
data = {
    "slidingWindow": SLIDING_WINDOW,
    "alignPolicy": {
        "alignMode": "Outer",
        "fillNAMethod": "Linear", 
        "paddingValue": 0
    },
    "dataSource": DATA_SOURCE_URL,
    "dataSchema": "MultiTable",
    "startTime": "2021-01-01T00:00:00Z", 
    "endTime": "2021-01-02T09:19:00Z", 
    "displayName": "SampleRequest"
}

res = requests.post(API_MODEL.format(endpoint=ENDPOINT), data=json.dumps(data), headers=HEADERS)
assert res.status_code == 201, f"Error occured. Error message: {res.content}"
print(res.content)
location = res.headers['Location']
print(location)
model_id = location[location.rindex('/')+1:]
print(model_id)

b'{"modelId": "faf376fc-52c5-11ed-8ec8-2290545d1797", "createdTime": "2022-10-23T11:29:39Z", "lastUpdatedTime": "2022-10-23T11:29:39Z", "modelInfo": {"dataSource": "https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_3000", "dataSchema": "MultiTable", "startTime": "2021-01-01T00:00:00Z", "endTime": "2021-01-02T09:19:00Z", "displayName": "SampleRequest", "slidingWindow": 200, "alignPolicy": {"alignMode": "Outer", "fillNAMethod": "Linear", "paddingValue": 0.0}, "status": "CREATED", "errors": [], "diagnosticsInfo": {"modelState": {"epochIds": [], "trainLosses": [], "validationLosses": [], "latenciesInSeconds": []}, "variableStates": []}}}\n'
https://multi-ad-test-uscx.cognitiveservices.azure.com:443/anomalydetector/v1.1/multivariate/models/faf376fc-52c5-11ed-8ec8-2290545d1797
faf376fc-52c5-11ed-8ec8-2290545d1797


# Get model status

In [24]:
res = requests.get(API_MODEL_STATUS.format(endpoint=ENDPOINT, model_id = model_id), headers=HEADERS)
assert res.status_code == 200, f"Error occured. Error message: {res.content}"
res_content = json.loads(res.content)
print(json.dumps(res_content))
print(res_content['modelInfo']['status'])

{"modelId": "faf376fc-52c5-11ed-8ec8-2290545d1797", "createdTime": "2022-10-23T11:29:39Z", "lastUpdatedTime": "2022-10-23T11:33:15Z", "modelInfo": {"dataSource": "https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_3000", "dataSchema": "MultiTable", "startTime": "2021-01-01T00:00:00Z", "endTime": "2021-01-02T09:19:00Z", "displayName": "SampleRequest", "slidingWindow": 200, "alignPolicy": {"alignMode": "Outer", "fillNAMethod": "Linear", "paddingValue": 0.0}, "status": "READY", "errors": [], "diagnosticsInfo": {"modelState": {"epochIds": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], "trainLosses": [0.3033768121491779, 0.2433272126046094, 0.22904170710932126, 0.27909666841680353, 0.2263397140936418, 0.22412160580808466, 0.22453346035697241, 0.22147076644680716, 0.2214091630144553, 0.2234686863693324], "validationLosses": [0.2903098963727855, 0.26336782628839667, 0.2507747303355824, 0.3450460241298483, 0.2572712561096808, 0.24944635834356751, 0.25021902720133465, 0.24727

# Asynchronous Inference

## Trigger asynchronous inference

In [38]:
data = {
    "dataSource": DATA_SOURCE_URL,
    "topContributorCount": 10,
    "startTime": "2021-01-02T12:00:00Z", 
    "endTime": "2021-01-03T00:00:00Z"
}

res = requests.post(API_MODEL_BATCH_INFERENCE.format(endpoint=ENDPOINT, model_id=model_id), 
                    data=json.dumps(data), headers=HEADERS)

print(res.headers)
print(res.content)
assert res.status_code == 202, f"Error occured. Error message: {res.content}"
result_id = res.headers["operation-id"]
print(f"result id = {result_id}")

{'Content-Length': '348', 'Content-Type': 'application/json', 'Date': 'Sun, 23 Oct 2022 14:52:34 GMT', 'Server': 'istio-envoy', 'operation-location': 'https://multi-ad-test-uscx.cognitiveservices.azure.com:443/anomalydetector/v1.1/multivariate/detect-batch/', 'operation-id': '53be6eba-52e2-11ed-ac38-86116376b78c', 'x-envoy-upstream-service-time': '268', 'apim-request-id': '498d2634-0d86-4a31-bb3e-8b3eb075d255', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-ms-region': 'Central US EUAP'}
b'{"resultId": "53be6eba-52e2-11ed-ac38-86116376b78c", "summary": {"status": "CREATED", "errors": [], "variableStates": [], "setupInfo": {"dataSource": "https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_3000", "topContributorCount": 10, "startTime": "2021-01-02T12:00:00Z", "endTime": "2021-01-03T00:00:00Z"}}, "results": []}\n'
result id = 53be6eba-52e2-11ed-ac38-86116376b78c


## Get asynchronous inference results

In [39]:
res = requests.get(API_RESULTS.format(endpoint=ENDPOINT, result_id=result_id), headers=HEADERS)
assert res.status_code == 200, f"Error occured. Error message: {res.content}"
print(res.content)

b'{"resultId": "53be6eba-52e2-11ed-ac38-86116376b78c", "summary": {"status": "READY", "errors": [], "variableStates": [{"variable": "series_0", "filledNARatio": 0.0, "effectiveCount": 721, "firstTimestamp": "2021-01-02T12:00:00Z", "lastTimestamp": "2021-01-03T00:00:00Z"}, {"variable": "series_1", "filledNARatio": 0.0, "effectiveCount": 721, "firstTimestamp": "2021-01-02T12:00:00Z", "lastTimestamp": "2021-01-03T00:00:00Z"}, {"variable": "series_2", "filledNARatio": 0.0, "effectiveCount": 721, "firstTimestamp": "2021-01-02T12:00:00Z", "lastTimestamp": "2021-01-03T00:00:00Z"}, {"variable": "series_3", "filledNARatio": 0.0, "effectiveCount": 721, "firstTimestamp": "2021-01-02T12:00:00Z", "lastTimestamp": "2021-01-03T00:00:00Z"}, {"variable": "series_4", "filledNARatio": 0.0, "effectiveCount": 721, "firstTimestamp": "2021-01-02T12:00:00Z", "lastTimestamp": "2021-01-03T00:00:00Z"}], "setupInfo": {"dataSource": "https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_3000", 

# Synchronous Inference

In [49]:
import json

with open("5_3000.json", "r") as f:
    sample_input = json.load(f)

url = API_MODEL_LAST_INFERENCE.format(endpoint=ENDPOINT,model_id=model_id)
payload = sample_input

res = requests.post(url, headers=HEADERS, data=json.dumps(payload))
print(res.content)

b'{"variableStates": [{"variable": "series_0", "filledNARatio": 0.0, "effectiveCount": 1, "firstTimestamp": "2021-01-03T01:59:00Z", "lastTimestamp": "2021-01-03T01:59:00Z"}, {"variable": "series_1", "filledNARatio": 0.0, "effectiveCount": 1, "firstTimestamp": "2021-01-03T01:59:00Z", "lastTimestamp": "2021-01-03T01:59:00Z"}, {"variable": "series_2", "filledNARatio": 0.0, "effectiveCount": 1, "firstTimestamp": "2021-01-03T01:59:00Z", "lastTimestamp": "2021-01-03T01:59:00Z"}, {"variable": "series_3", "filledNARatio": 0.0, "effectiveCount": 1, "firstTimestamp": "2021-01-03T01:59:00Z", "lastTimestamp": "2021-01-03T01:59:00Z"}, {"variable": "series_4", "filledNARatio": 0.0, "effectiveCount": 1, "firstTimestamp": "2021-01-03T01:59:00Z", "lastTimestamp": "2021-01-03T01:59:00Z"}], "results": [{"timestamp": "2021-01-03T01:59:00Z", "value": {"isAnomaly": false, "severity": 0.15577223896980286, "score": 0.26766061782836914, "interpretation": []}, "errors": []}]}\n'


# List models

In [52]:
res = requests.get(API_MODEL.format(endpoint=ENDPOINT), headers=HEADERS)
assert res.status_code == 200, f"Error occured. Error message: {res.content}"
print(res.content)

b'{"models": [{"modelId": "a0ea52da-52c9-11ed-8ec8-2290545d1797", "createdTime": "2022-10-23T11:55:46Z", "lastUpdatedTime": "2022-10-23T11:59:27Z", "modelInfo": {"dataSource": "https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_3000", "dataSchema": "MultiTable", "startTime": "2021-01-01T00:00:00Z", "endTime": "2021-01-02T09:19:00Z", "displayName": "SampleRequest", "slidingWindow": 200, "alignPolicy": {"alignMode": "Outer", "fillNAMethod": "Linear", "paddingValue": 0.0}, "status": "READY", "errors": [], "diagnosticsInfo": {"modelState": {"epochIds": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], "trainLosses": [0.3033768121491779, 0.2433272126046094, 0.22904170710932126, 0.27909666841680353, 0.2263397140936418, 0.22412160580808466, 0.22453346035697241, 0.22147076644680716, 0.2214091630144553, 0.2234686863693324], "validationLosses": [0.2903098963727855, 0.26336782628839667, 0.2507747303355824, 0.3450460241298483, 0.2572712561096808, 0.24944635834356751, 0.250219027201

# Delete model

In [51]:
res = requests.delete(API_DELETE.format(endpoint=ENDPOINT, model_id=model_id), headers=HEADERS)
assert res.status_code == 204, f"Error occured. Error message: {res.content}"
print(res.content)

b''
